# Adversarial Chess
6.4212 Final Project

Kameron Dawson

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (
    StartMeshcat, Simulator, Rgba, RigidTransform
)
from pydrake.geometry import Sphere
from manipulation.utils import RenderDiagram
from setup.setup import setup
from utils import visualize_box, visualize_text, visualize_iiwa_reachability
from optimization.utils import find_home_configuration
from control.controller import Controller

In [ ]:
# Setup meshcat for visualization
meshcat = StartMeshcat()
print('Click the link above to open Meshcat in your browser!')

In [ ]:
# Clear meshcat
meshcat.Delete()

In [ ]:
# Setup the scene
diagram, station = setup(meshcat)

In [ ]:
# Make sure the system is working
RenderDiagram(diagram, max_depth=1)

In [ ]:
# Get relevant vars from the diagram, station
context = diagram.CreateDefaultContext()
plant = station.plant()
plant_context = diagram.GetSubsystemContext(plant, context)

# Get controllers
iiwa1_traj_controller = diagram.GetSubsystemByName('iiwa1_traj_controller')
iiwa2_traj_controller = diagram.GetSubsystemByName('iiwa2_traj_controller')
iiwa1_grasp_controller = diagram.GetSubsystemByName('iiwa1_grasp_controller')
iiwa2_grasp_controller = diagram.GetSubsystemByName('iiwa2_grasp_controller')

# Get iiwa plants
plant_iiwa1, plant_iiwa2 = iiwa1_traj_controller._plant, iiwa2_traj_controller._plant

# Get plant contexts
plant_context_iiwa1, plant_context_iiwa2 = iiwa1_traj_controller._plant_context, iiwa2_traj_controller._plant_context

In [ ]:
# Create and run a simulator
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(0.1)

In [ ]:
# visualize_iiwa_reachability(meshcat, plant_iiwa2, N=1000)

In [ ]:
# home, perf = find_home_configuration(plant_iiwa1, plant_context_iiwa1, n_candidates=10000, pos_tol=0.05, rot_tol=0.05)
# print('Home configuration: ', home)
# print('Relative improvement: ', perf)

In [ ]:
# Setup the overall controller
controller = Controller(diagram, context, simulator, iiwa1_traj_controller, iiwa2_traj_controller, iiwa1_grasp_controller, iiwa2_grasp_controller)

## Perception

### Sanity Check Cameras

In [ ]:
# Run the following cell to visualize the rgb outputs of each of the cameras
cameras = ["camera0", "camera1", "camera2"]
station_context = diagram.GetSubsystemContext(station, context)

fig, axes = plt.subplots(
    1, len(cameras), figsize=(5 * len(cameras), 4), constrained_layout=True
)
for ax, cam in zip(axes, cameras):
    img = station.GetOutputPort(f"{cam}.rgb_image").Eval(station_context)
    arr = np.array(img.data, copy=False).reshape(img.height(), img.width(), -1)
    im = ax.imshow(arr)
    ax.set_title(f"{cam} rgb image")
    ax.axis("off")

plt.show()

In [ ]:
# Run the following cell to visualize the depth outputs of each of the cameras
fig, axes = plt.subplots(
    1, len(cameras), figsize=(5 * len(cameras), 4), constrained_layout=True
)
for ax, cam in zip(axes, cameras):
    img = station.GetOutputPort(f"{cam}.depth_image").Eval(station_context)
    depth_img = np.array(img.data, copy=False).reshape(img.height(), img.width(), -1)
    depth_img = np.ma.masked_invalid(depth_img)
    img = ax.imshow(depth_img, cmap="magma")
    ax.set_title(f"{cam} depth image")
    ax.axis("off")

plt.show()

### Sanity Check Perception

In [ ]:
# Get poses of every chess piece
piece_poses = controller.get_piece_poses()

In [ ]:
# Visualize poses as sanity check
colors = {
    'p': Rgba(1, 0, 0), # red
    'k': Rgba(0, 1, 0), # green
    'q': Rgba(0, 0, 1), # blue
    'b': Rgba(1, 1, 0), # yellow
    'n': Rgba(0, 1, 1), # cyan
    'r': Rgba(1, 0, 1) # magenta
}

for piece, poses in piece_poses.items():
    for i, X in enumerate(poses):
        path = f'/piece/{piece}{i}'
        meshcat.SetTransform(path, X)
        meshcat.SetObject(
            path, Sphere(0.01), rgba=colors[piece.lower()]
        )

In [ ]:
from perception.perception import perception

perception_result = perception(controller.diagram, controller.context, controller.oriented_model_piece_point_clouds, debugging=True)
# Visualize piece clouds
piece_pcs = perception_result['piece_pcs']
for color in piece_pcs:
    for piece, pc in piece_pcs[color].items():
        rgb = Rgba(1, 1, 1) if color == 'light' else Rgba(0, 0, 0)
        meshcat.SetObject(
            f'piece_pc_{color}_{piece}', pc, point_size=0.0025, rgba=rgb
        )

# Visualize piece bounding boxes
for color in piece_pcs:
    for piece, pc in piece_pcs[color].items():
        pts = pc.xyzs()
        x, y, z = pts[0], pts[1], pts[2]
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        min_z, max_z = np.min(z), np.max(z)
        lower, upper = np.array([min_x, min_y, min_z]), np.array([max_x, max_y, max_z])
        visualize_box(meshcat, lower, upper, f'piece_bb_{color}_{piece}_{i}')

## Motion Planning

In [ ]:
controller.control_loop(None)

In [ ]:
X = RigidTransform()
X.set_translation([-0.16449999999999998, 0.07050000000000001, 0.527262])
path = '/place'
meshcat.SetTransform(path, X)
meshcat.SetObject(
    path, Sphere(0.01), rgba=Rgba(1, 0, 0)
)

In [ ]:
from pydrake.all import (
    RollPitchYaw, RotationMatrix
)
from game.utils import Game

game = Game()
for file_idx in range(ord('a'), ord('h') + 1):
        file = chr(file_idx)
        for rank_idx in range(1, 8 + 1):
            rank = str(rank_idx)
            sq = file + rank
            X_WG_pick = game.square_to_pose(sq)
            rpy_down = RotationMatrix(RollPitchYaw(-np.pi/2, 0, 0))
            pick_xyz = X_WG_pick.translation()
            X_WG_prepick = RigidTransform(rpy_down, [pick_xyz[0], pick_xyz[1], pick_xyz[2] + 0.1 + 2*0.076])
            meshcat.SetTransform(sq, X_WG_prepick)
            meshcat.SetObject(
                sq, Sphere(0.01), rgba=Rgba(1, 0, 0)
            )


In [ ]:
X_table = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("link", plant.GetModelInstanceByName('table')))
print("Table:", X_table.translation())

X_board = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("link", plant.GetModelInstanceByName('chessboard')))
print("Chessboard:", X_board.translation())

In [ ]:


controller.chess_move(1, )